##Connect to database! 
###Get Neighborhood Data

In [5]:
#send_Example.py 
import pandas as pd
import json
import psycopg2
import random

###Get tweets wtihin california, and shapes with the county of SF ('075')

In [14]:
conn_dict = {'dbname':'zipfiantwitter', 'user':'clwilloughby', 'password': '', 'host':'/tmp'}
conn = psycopg2.connect(dbname=conn_dict['dbname'], user=conn_dict['user'], host='/tmp')
cur = conn.cursor()

map_query = """
        SELECT
                DISTINCT
                ST_asGEOJSON(wkb_geometry) geometry
                , geoid10

                FROM sf_neighb 
                WHERE countyfp10 = '075'
                ;"""

df = pd.read_sql(map_query, conn)

In [15]:
#intermediate step: not needed normally
#df.to_csv('sf_only_sql_shapes.csv')
#df = pd.read_csv('sql_shapes.csv')

In [16]:
df.head()

,geometry,geoid10
0,"{""type"":""MultiPolygon"",""coordinates"":[[[[-122....",060750179021009
1,"{""type"":""MultiPolygon"",""coordinates"":[[[[-122....",060759806001043
2,"{""type"":""MultiPolygon"",""coordinates"":[[[[-122....",060759806001041
3,"{""type"":""MultiPolygon"",""coordinates"":[[[[-122....",060759806001038
4,"{""type"":""MultiPolygon"",""coordinates"":[[[[-122....",060750179021038


###Get a Random Color For Each Neighborhood
####Later will encode information by color and hue

In [17]:
def getrandom_color(df, color_file):
    '''takes in a dataframe of gemoetry and geoid, and a list of hex colors
    Creates a new colomn of shape colors. Returns the new dataframe.'''
    #import the colors csv
    cdf = pd.read_csv('colors.csv')
    #make it a list
    colors = cdf[' color'].tolist() 
    num_col = df.shape[0] 
    df['hexcolors'] = [random.choice(colors) for i in xrange(num_col)]
    return df

In [18]:
df_col = getrandom_color(df, 'colors.csv')


In [19]:
df_col.head()


,geometry,geoid10,hexcolors
0,"{""type"":""MultiPolygon"",""coordinates"":[[[[-122....",060750179021009,#66FF00
1,"{""type"":""MultiPolygon"",""coordinates"":[[[[-122....",060759806001043,#66FFFF
2,"{""type"":""MultiPolygon"",""coordinates"":[[[[-122....",060759806001041,#CC9966
3,"{""type"":""MultiPolygon"",""coordinates"":[[[[-122....",060759806001038,#CC6600
4,"{""type"":""MultiPolygon"",""coordinates"":[[[[-122....",060750179021038,#99FF33


###Convert DataFrame to JsonFile for Mapping

In [20]:
def add_properties_geo(row):
    geoid = row['geoid10']
    hexcolor = row['hexcolors']
    geo_json = {"type": "Feature", "geometry": json.loads(row['geometry']),  "properties": {'geoid': geoid , 'hexcolor' : hexcolor }}
    return geo_json

list_to_export = [add_properties_geo(row) for idx, row in df_col.iterrows()]

with open('sf_geo.json', 'w') as outfile:
    json.dump(list_to_export, outfile)